# Explore here

It's recommended to use this notebook for exploration purposes.

In [4]:
#dont forget to: pip install lxml otherwise code will not run
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

#storing URL
tesla_url='https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue'

#Creating request
time.sleep(20) #20 sec delay before making the request to not overload server
try:
    response=requests.get(tesla_url)
    response.raise_for_status()
    tesla_raw=BeautifulSoup(response.text, 'html')

#handles errors specific to the request 
except requests.exceptions.RequestException as e:
  print(f"Request Exception: {e}") #prints the error
  header={'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
  response=requests.get(tesla_url, headers=header)
  tesla_raw=BeautifulSoup(response.text, 'html')

# Prints other exceptions that may occur during parsing or processing
except Exception as e:
    print(f"Other Exception: {e}")

Request Exception: 403 Client Error: Forbidden for url: https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue


In [8]:
#finding all tables about Tesla's financial performance
all_tables = tesla_raw.find_all("table")

#finding quarterly evolution table of Tesla's revenue
for table in all_tables:
    if 'Tesla Quarterly Revenue' in table.get_text():
      tesla_data=table
      break #stops the code once it finds the table

#storing result as dataframe
tesla_qrev=pd.read_html(str(tesla_data))[0]
tesla_qrev.columns=['quater', 'revenue'] #renames columns
tesla_qrev['quater']=pd.to_datetime(tesla_qrev['quater']) #sets date format
tesla_qrev['revenue']=tesla_qrev['revenue'].str.replace('[\$,]','', regex=True).astype(float) #removes $ and , 

tesla_qrev

ImportError: lxml not found, please install it

In [ ]:
import sqlite3

con = sqlite3.connect("tesla_qrev.db")
cursor=con.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS TESLA
    (quater        DATE    NOT NULL,
     revenuq         FLOAT)""")

tesla_qrev.to_sql('TESLA', con, index=False, if_exists='replace')

#Checking the table was created properly
for row in cursor.execute("SELECT * FROM TESLA"):
    print(row)

con.commit()
con.close